In [1]:
import pandas as pd
import os

In [2]:
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID

In [3]:
teams_df = pd.DataFrame([])
current_df = pd.DataFrame([])
for team_file in os.listdir('data/TeamsPrep'):
    team_id = team_file[:3]
    
    for year in [2020, 2021, 2022, 2023]:
        team_df = pd.read_excel(f'data/TeamsPrep/{team_file}', str(year))

        team_df['TeamID'] = mapTeamID(team_id)
        team_df['OppID'] = team_df['OppID'].apply(lambda x: mapTeamID(x))
        team_df['Season'] = year

        teams_df = pd.concat([teams_df, team_df])
        
    team_df = pd.read_excel(f'data/CurrentSeason/{team_file}')
    
    roll_df = getTeamRollingSeason(team_df)
    prep_df = dataPrep(roll_df, 'Games')
    
    prep_df['TeamID'] = mapTeamID(team_id)
    prep_df['OppID'] = prep_df['OppID'].apply(lambda x: mapTeamID(x))
    prep_df['Season'] = 2024
    
    current_df = pd.concat([current_df, prep_df])
    
teams_df = teams_df.drop(columns=['Date']).reset_index(drop=True)
current_df = current_df.drop(columns=['Date']).reset_index(drop=True)

In [4]:
teams_df.to_excel('Models/train.xlsx', index=0)
current_df.to_excel('Models/test.xlsx', index=0)